```{contents}
```
## Non-Deterministic Execution

**Non-deterministic execution** in LangGraph refers to workflows where **the same input state can lead to different execution paths, decisions, or outputs across runs**, due to stochastic components (LLMs, sampling), dynamic routing, concurrency, or external side effects.
This property is **fundamental** for building adaptive, agentic, and real-world AI systems.

---

### **1. Deterministic vs Non-Deterministic**

| Aspect                   | Deterministic     | Non-Deterministic            |
| ------------------------ | ----------------- | ---------------------------- |
| Same input → same output | ✅                 | ❌                            |
| Execution path           | Fixed             | Data-dependent               |
| LLM sampling             | Disabled / greedy | Enabled (temperature, top-p) |
| Routing                  | Static            | Dynamic                      |
| External tools           | No                | Yes                          |
| Concurrency              | No                | Yes                          |

LangGraph **explicitly supports non-determinism** while preserving **control, observability, and recoverability**.

---

### **2. Sources of Non-Determinism in LangGraph**

#### **A. LLM Sampling**

```python
llm = ChatOpenAI(temperature=0.8)
```

Different tokens → different reasoning → different downstream routing.

#### **B. Dynamic Conditional Routing**

```python
def route(state):
    if "search" in state["plan"]:
        return "search_node"
    return "final_node"
```

LLM output controls the next node.

#### **C. Tool & Environment Feedback**

APIs, databases, filesystems, and sensors introduce unpredictable responses.

#### **D. Parallel & Async Execution**

Task completion order may vary.

#### **E. Human-in-the-Loop**

Human input is inherently non-deterministic.

---

### **3. How LangGraph Controls Non-Determinism**

LangGraph does **not** remove non-determinism.
It **contains and manages** it.

| Mechanism          | Purpose                   |
| ------------------ | ------------------------- |
| Typed State        | Structural consistency    |
| Checkpointing      | Save every step           |
| Replay             | Reproduce past runs       |
| Conditional Guards | Prevent invalid paths     |
| Recursion Limits   | Prevent runaway loops     |
| Tracing            | Full execution visibility |

---

### **4. Minimal Non-Deterministic Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict
import random

class State(TypedDict):
    value: int

def stochastic_node(state):
    return {"value": state["value"] + random.randint(1, 3)}

def router(state):
    if state["value"] >= 10:
        return END
    return "stochastic"

builder = StateGraph(State)
builder.add_node("stochastic", stochastic_node)
builder.set_entry_point("stochastic")
builder.add_conditional_edges("stochastic", router, {
    "stochastic": "stochastic",
    END: END
})

graph = builder.compile()

print(graph.invoke({"value": 0}))
```

Each execution produces a **different path length and result**.

---

### **5. Non-Determinism with LLM Agents**

```python
def agent(state):
    response = llm.invoke(state["query"])
    return {"decision": response.content}
```

Combined with:

```python
def route(state):
    if "tool" in state["decision"]:
        return "tool_node"
    return "final"
```

This produces **probabilistic agent behavior** — essential for:

* Planning
* Exploration
* Self-correction
* Creativity

---

### **6. Production Risks & Controls**

| Risk                 | Control                 |
| -------------------- | ----------------------- |
| Unbounded loops      | `recursion_limit`       |
| Unexpected routes    | Conditional guards      |
| High cost            | Budget + token tracking |
| Debugging difficulty | Checkpoints + tracing   |
| Compliance issues    | Human approval gates    |

```python
graph.invoke(input, config={"recursion_limit": 20})
```

---

### **7. Why Non-Determinism is Necessary**

| Capability            | Requires Non-Determinism |
| --------------------- | ------------------------ |
| Autonomous agents     | ✅                        |
| Multi-step reasoning  | ✅                        |
| Exploration           | ✅                        |
| Robust adaptation     | ✅                        |
| Real-world tool usage | ✅                        |

A deterministic system **cannot adapt** to uncertain environments.

---

### **8. Mental Model**

LangGraph treats non-determinism as:

> **A controlled source of exploration wrapped inside a formally verifiable execution engine.**

This is the foundation of **safe autonomous systems**.


In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict
import random

# -------------------------
# Define shared state
# -------------------------
class State(TypedDict):
    total: int
    history: list

# -------------------------
# Stochastic node
# -------------------------
def roll(state: State) -> State:
    value = random.randint(1, 6)  # dice roll
    return {
        "total": state["total"] + value,
        "history": state["history"] + [value]
    }

# -------------------------
# Router: stop when total >= 20
# -------------------------
def decide(state: State):
    if state["total"] >= 20:
        return END
    return "roll"

# -------------------------
# Build graph
# -------------------------
builder = StateGraph(State)
builder.add_node("roll", roll)
builder.set_entry_point("roll")

builder.add_conditional_edges(
    "roll",
    decide,
    {"roll": "roll", END: END}
)

graph = builder.compile()

In [2]:
for i in range(3):
    result = graph.invoke({"total": 0, "history": []})
    print(f"Run {i+1}: {result}")


Run 1: {'total': 20, 'history': [5, 5, 1, 2, 2, 5]}
Run 2: {'total': 20, 'history': [1, 4, 1, 1, 1, 1, 1, 5, 5]}
Run 3: {'total': 23, 'history': [5, 6, 5, 2, 1, 4]}
